# Experiments with Dense Vectors

In [1]:
import os
import sys
import numpy as np
from numba import jit
sys.path.append(os.path.abspath('../')) # needed to import from permutations sibling directory
from permutations import dense_permutations as p

In [2]:
# Generate some vectors
a = p.get_random_vector(100)
b = p.get_random_vector(100)
c = p.get_random_vector(100)

# Generate some predicates
p_a = p.get_random_permutation(100)
p_b = p.get_random_permutation(100)
p_c = p.get_random_permutation(100)

In [8]:
# In theory, this should give us the ability to deduce a permutation from two source vectors; untested
def deduceperm(permuted, unpermuted): 
     return np.argsort(np.argsort(permuted)[np.argsort(np.argsort(unpermuted))]) 
    

# Changes a permutation to be like another permutation, where howmuch is the number of dimensions the input perm
# should have in common with tobelike. In theory it should never become disjoint with range(dimension), but I haven't
# rigorously tested it. Assuming it's fed with a permutation derived from range(dimension), that is.
def trainpermutation(perm, tobelike, howmuch):
    likeness = np.sum(perm == tobelike)
    perm1 = np.copy(perm) 
    while likeness < howmuch: 
        tochange = np.random.choice(np.where(perm1 != tobelike)[0]) 
        perm1[np.where(perm1 == tobelike[tochange])] = perm1[tochange] 
        perm1[tochange] = tobelike[tochange] 
        likeness = np.sum(perm1 == tobelike) 
    return perm1        

In [4]:
p_ta = trainpermutation(p.get_random_permutation(100), p_a, 50) # make a permutation with 50 dimensions in common with p_a
print(np.setdiff1d(np.arange(100), p_ta)) # should be an empty list
print(np.sum(p_ta == p_a)) # should be precisely 50

[]
50


In [7]:
print(p.cosine_similarity(a[p_a], a[p_b])) # basically random
print(p.cosine_similarity(a[p_ta], a[p_a])) # same vector, more similar permutations, should be more similar
print(p.cosine_similarity(a,b))
print(p.cosine_similarity(a[p_a], b[p_a])) # dissimilar vectors, same permutation, should be (identically to unpermuted) dissimilar

0.5309507
0.68518347
0.44493514
0.44493514


In [11]:
np.sum(deduceperm(a[p_a], a) == p_a) # should be 100 (e.g. every dimension should match)

100

## Training Things
If we want to train, we should go for something like `sigmoid(vec[permutation].dot(vec2))`

In [12]:
def sigmoid(x):
    return np.exp(-np.logaddexp(0, -x))

In [15]:
sigmoid(a[p_a].dot(c))

0.9448489752648982